In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
home_dir = "../"

import pandas as pd

In [4]:
task="pmd"
result_df = pd.read_csv(home_dir+f"models/dbnsfp/outputs_postprocessed/{task}.tsv", sep="\t")
print(result_df.columns)
print(result_df.shape)
result_df["class"].value_counts()

Index(['mut_id', 'md5', 'pmd_id', 'nr', 'prot_acc_version', 'snp_id',
       'mut_real', 'wt', 'mut', 'prot_pos', 'chrom', 'chrom_pos', 'ref_allele',
       'alt_allele', 'function', 'source', 'crossref', 'function_summarized',
       'class', 'seq', 'SIFT_score', 'Polyphen2_HVAR_score', 'MetaRNN_score',
       'REVEL_score', 'MVP_score', 'CADD_raw_score',
       'integrated_fitCons_score', 'phyloP17way_primate_score',
       'phastCons17way_primate_score', 'bStatistic_score'],
      dtype='object')
(7179, 30)


Effect       3818
No-effect    1777
Knock-out    1584
Name: class, dtype: int64

In [7]:
col_renaming_dict = {"prot_pos":"1indexed_prot_mt_pos", "wt":'wt_aa_1letter', "mut":'mt_aa_1letter', 'chrom': "chrom_num"}
result_renamed_df = result_df.rename(columns=col_renaming_dict)
print(result_renamed_df.columns)

Index(['mut_id', 'md5', 'pmd_id', 'nr', 'prot_acc_version', 'snp_id',
       'mut_real', 'wt_aa_1letter', 'mt_aa_1letter', '1indexed_prot_mt_pos',
       'chrom_num', 'chrom_pos', 'ref_allele', 'alt_allele', 'function',
       'source', 'crossref', 'function_summarized', 'class', 'seq',
       'SIFT_score', 'Polyphen2_HVAR_score', 'MetaRNN_score', 'REVEL_score',
       'MVP_score', 'CADD_raw_score', 'integrated_fitCons_score',
       'phyloP17way_primate_score', 'phastCons17way_primate_score',
       'bStatistic_score'],
      dtype='object')


In [9]:
# this copy is saving for running the other methods.
out_filepath = home_dir+f"data/datasets_pmd/{task}_dbnsfp"

# Creating merged fasta document ...
sequences_df = result_renamed_df[['prot_acc_version', 'crossref', 'seq']].drop_duplicates(keep="first")
print("#-of sequences", sequences_df.shape[0])

out_fasta_filepath = out_filepath+".fasta"
out_fasta_file_handle = open(out_fasta_filepath, "w")

for tuple in sequences_df.itertuples():
    out_fasta_file_handle.write(f">{tuple.prot_acc_version} | {tuple.crossref}\n")
    out_fasta_file_handle.write(f"{tuple.seq}\n")
    # break
out_fasta_file_handle.close() 

result_renamed_df.drop(columns=["seq"], inplace=True)
result_renamed_df.to_csv(out_filepath+".tsv", sep="\t", index=False)

#-of sequences 2056
